In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from pandas import ExcelWriter

import math
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import f1_score, accuracy_score
import os



# !pip install pytorch-tabnet
import pandas as pd
import numpy  as np
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold

os.chdir("C:/Users/VIP13/Статья 2/Котировки 6")

In [ ]:
frame_3=pd.read_csv('all_snp_btc_rv_rr.csv', delimiter=';')
frame=frame_3.iloc[:,1:].copy() #на входе подаются квадраты волатильностей, поэтому сначала из них берем корни, а потом логарифмируем
frame=frame.loc[:, [c for c in frame.columns if (frame[c]>0).all()]]
frame=np.log(np.sqrt(frame))
ind=[i for i in range(0,len(frame))]
frame.index=ind
frame.head()
data=frame['rv_shk_snp']
# data=frame['rv_shk_btc']

In [ ]:
def split_sequence(sequence, look_back, forecast_horizon):
 X, y = list(), list()
 for i in range(len(sequence)): 
   lag_end = i + look_back
   forecast_end = lag_end + forecast_horizon
   if forecast_end > len(sequence):
     break
   seq_x, seq_y = sequence[i:lag_end], sequence[lag_end:forecast_end]
   X.append(seq_x)
   y.append(seq_y)
 return np.array(X), np.array(y)

In [ ]:
from pytorch_tabnet.augmentations import RegressionSMOTE
aug = RegressionSMOTE(p=0.2)
prog=[]
orig=[]


step_smooth=5
len_wind=399
iteratation_5=int((len(data)-len_wind)/step_smooth)+1


for i in range(0,iteratation_5):
    
    train=data.iloc[0+step_smooth*i:len_wind+step_smooth*i]
    test=data.iloc[0+step_smooth*i:len_wind+1+step_smooth*i]
    
    x_train, y_train=split_sequence(train,5,1)
    x_test, y_test=split_sequence(test,5,1)
    
    regressor = TabNetRegressor(verbose=0,seed=7,  scheduler_params={"step_size":10, # how to use learning rate scheduler
                                         "gamma":0.9})
    regressor.fit(X_train=x_train, y_train=y_train,
#                   eval_set=y_train,
                  patience=10, max_epochs=200,
                  eval_metric=['mae'],
                 batch_size=16, virtual_batch_size=16,
    num_workers=0,
    drop_last=False,
                 )
    prog.append(regressor.predict(x_test)[-1][0])
    orig.append(y_test[-1][0])
    print(i)

In [ ]:
print((abs(np.exp(orig)-np.exp(prog))/np.exp(orig)).mean())
